In [1]:
## 导入库和config
import csv
import random
import numpy
from utls import shuffle_and_split,Bag

random.seed(42)
numpy.random.seed(42)

USE_PHRASE = False # 只使用句子还是也使用句子片段

In [2]:
## 读取数据，并简单处理
with open(r'.\data\train.tsv') as f:
    tsvreader = csv.reader(f, delimiter='\t')
    temp = list(tsvreader)
print(temp[:5])
print(temp[-5:])

data = temp[1:]
if not USE_PHRASE:
    data = [data[i] for i in range(len(data)) 
            if (i==0 or data[i][1]!=data[i-1][1]) ] 
print(*data[:5],sep='\n')

for i in range(len(data)):
    data[i][1] = int(data[i][1])
    data[i][3] = int(data[i][3])
print(*data[:5],sep='\n')    

[['PhraseId', 'SentenceId', 'Phrase', 'Sentiment'], ['1', '1', 'A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .', '1'], ['2', '1', 'A series of escapades demonstrating the adage that what is good for the goose', '2'], ['3', '1', 'A series', '2'], ['4', '1', 'A', '2']]
[['156056', '8544', "Hearst 's", '2'], ['156057', '8544', 'forced avuncular chortles', '1'], ['156058', '8544', 'avuncular chortles', '3'], ['156059', '8544', 'avuncular', '2'], ['156060', '8544', 'chortles', '2']]
['1', '1', 'A series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .', '1']
['64', '2', 'This quiet , introspective and entertaining independent is worth seeking .', '4']
['82', '3', "Even fans of Ismail Merchant 's work , I suspect , would ha

In [3]:
## 使用词袋模型


sentences = [idata[2] for idata in data]

vocab = Bag(sentences)
vocab.init_vocab()
inputlen = vocab.get_vocab_len()

6823 1706


In [ ]:
## 模型



## dataloader
train_rate = 0.8
batch_size = 16

train_data, valid_data = shuffle_and_split(data,TRAIN_RATE)
print(len(train_data),len(valid_data))

class dataloader(vocab,train_data,mode='train'):
    if mode == 'train':